<a href="https://colab.research.google.com/github/LuizJoca/shoreline_analisys_protocol/blob/main/Extra%C3%A7%C3%A3o_Semi_Autom%C3%A1tica_de_Linha_de_Costa_ptbr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color = khaki>**Extração Semi-Automática de Linha de Costa**</font>

**Versão 0.1.0**

**Elaborado por: Luiz Henrique Joca Leite, Granduando em Geologia pela Universidade Federal do Ceará**

[Linkedin](https://www.linkedin.com/in/luiz-henrique-joca/) | [Github](https://github.com/LuizJoca)

Esse *notebook* faz parte dos produtos gerados pelo projeto de trabalho de conclusão de curso **"Ferramentas de Machine Learning Aplicadas em Prognóstico Geoambiental: Estudo de Caso da Linha de Costa do Estado do Ceará, NE Brasileiro"**. 

A metodologia aplicada neste *notebook* se baseia nos trabalhos de [Sousa *et al.,* (2018)](https://www.tandfonline.com/doi/abs/10.1080/01431161.2018.1455240) e [Ghorai e Mahapatra, (2020)](https://link.springer.com/article/10.1007/s41976-019-00030-w) e tem como principal objetivo gerar um raster binarizado para extração semi-automática das linhas de costa.

O código abaixo realiza as seguinte tarefas:
*   Aplicação de um filtro de mediana sobre imagens MNDWI pré-tratadas;
*   Simplificação do dado via converção para *8bit integer*;
*   Binarização via *Threshold*;
*   Exportação do raster binarizado para extração via GIS;

---


## <font color = khaki>**Preparação de Ambiente**</font>

Nessa etapa são  carregadas: 

*   As bibliotecas e pacotes necessários para execução do código;
*   O arquivo base que será manipulado no decorrer das etapas para binarização do raster e geração da linha de costa.

---

a) Caso o arquivo esteja sendo executado em Jupyter Notebook (nativo do computador):

No item 'Importação das Imagens Pré-Tratadas' a variavel `img_flaash` está com atribuição vazia. Após o sinal de `=` deve-se atribuir o caminho para o arquivo a ser utilizado entre aspas simples ou aspas duplas. Caso o arquivo esteja na mesma pasta do arquivo `.ipynb` basta adicionar o nome do arquivo, também entre aspas simples ou aspas duplas.

Feito isso basta executar o resto do notebook em sequência.  

b) Caso o arquivo esteja sendo executado em Google Colab:

O arquivo a ser utilizado precisa ser préviamente upando na aba de arquivos localizada na parte superior esquerda, ou deve estar presente em uma pasta do Google Drive linkada pelo mesmo e-mail utilizado na conta do Google Colab.

Para adição de pastas no google colab é necessário adicionar uma célular de código abaixo e adicionar o seguinte trecho:

      `from google.colab import drive`
      `drive.mount('/content/drive/')`

Isso permitira o link aos arquivos presentes no google drive do usuário. 

No item 'Importação das Imagens Pré-Tratadas' a variavel `img_flaash` está com atribuição vazia. Após o sinal de `=` deve-se atribuir o nome do arquivo, caso tenha sido upado na aba de arquivos. Já para casos no qual se tenha concedido acesso ao google drive basta copiar o caminho do arquivo a ser utilizado. Ambos os processos deve constar entre aspas simples ou aspas duplas. 

---

**Obs¹:** Os tratamentos realizados na imagem pelo autor na elaboração do trabalho de origem deste notebook foram realizadas no *software* ENVI e consistem nos processos de **Calibração Radiométrica**, **Calibração Atmosférica FLAASH** e **elaboração do MNDWI**. Sendo assim o autor recomenda a realização dos mesmo procedimentos para aquisição de resultados mais próximos aos adquiridos na pesquisa. 

### <font color = darkorange>Importando as bibliotecas</font>

In [ ]:
# Importação das Bibliotecas

import cv2
import numpy as np

from osgeo import osr
from osgeo import gdal
from osgeo import gdal_array

from skimage import io
from skimage import img_as_ubyte
from skimage.filters import median
from skimage.morphology import disk

from matplotlib import pyplot as plt

### <font color = darkorange>Importação das Imagens Pré-Tratadas</font>

In [ ]:
img_flaash =

In [ ]:
ds = gdal.Open(img_flaash)
gt = ds.GetGeoTransform()
proj = ds.GetProjection()

In [ ]:
mndwi =  ds.GetRasterBand(1)

In [ ]:
mndwi_array = mndwi.ReadAsArray()

In [ ]:
plt.figure()
plt.imshow(mndwi_array)
plt.colorbar()

## <font color = khaki>**Aplicação do Filtro de Mediana**</font>

In [ ]:
mndwi_median = median(mndwi_array, disk(3), mode = 'constant', cval = 0.0)

In [ ]:
plt.figure()
plt.imshow(mndwi_median)
plt.colorbar()

## <font color = khaki>**Conversão da Imagem para 8bit**</font>

In [ ]:
mndwi_8bit = img_as_ubyte(mndwi_median)

In [ ]:
mndwi_8bit.dtype

## <font color = khaki>**Aplicação do Threshold**</font>

In [ ]:
ret_mndwi, th_mndwi = cv2.threshold(mndwi_8bit, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

In [ ]:
plt.figure()
plt.imshow(th_mndwi)
plt.colorbar()

## <font color = khaki>**Exportação do Arquivo Binarizado**</font>

In [ ]:
driver = gdal.GetDriverByName('ENVI')
driver.Register()
outds = driver.Create('mndwi_threshold.dat',
                      xsize = th_mndwi.shape[1],
                      ysize = th_mndwi.shape[0],
                      bands = 1,
                      eType = gdal.GDT_Byte)
outds.SetGeoTransform(gt)
outds.SetProjection(proj)
outband = outds.GetRasterBand(1)
outband.WriteArray(th_mndwi)
outband.SetNoDataValue(0)
outband.FlushCache()

outband = None
outds = None